[Divide Two Integers](https://leetcode.com/problems/divide-two-integers/)。不使用乘除法实现整形的整除运算。

思路：乘法可以使用加法实现，反过来除法可以使用减法实现。被除数每减去一次除数，商就加1。这一过程可以优化，可以将除数翻倍，被除数减去除数的$n$倍，商就可以加上$n$，翻倍的过程可以使用位运算来实现。另外一个坑就是语言坑，由于一般的语言使用的是32位有符号整形补码，所以这里需要考虑表示范围：$[-2^{31},2^{31}-1]$。

In [22]:
def divide(dividend: int, divisor: int) -> int:
    res = 0    # 商
    sign = not ((dividend > 0) ^ (divisor > 0))    # 结果符号

    dividend, divisor = abs(dividend), abs(divisor)    # 转换成绝对值运算

    while dividend >= divisor:    # 只要被除数>除数就可以除
        n = 1
        a, b = dividend, divisor
        while a >= b*(n << 1):    # 被除数大于等于翻倍后的除数
            n <<= 1    # 所翻的倍数需要记住

        res += n
        dividend -= b*n

    if sign and res == 2**31:    # 正数越界
        res = res-1

    return res if sign else -res

2147483647

[String to Integer (atoi)](https://leetcode.com/problems/string-to-integer-atoi/)。实现str2int函数。

思路：该题需要考虑的情况如下：
- 字串两端的空格需要跳过，实际只需要跳过开头的空格
- 符号位只能出现在非空格字串的首位
- 遇到非法字符即刻返回
- 超出表示范围即刻返回

In [30]:
def myAtoi(s: str) -> int:
    head = True    # 首位标记
    sign = True
    res = 0

    for ch in s:
        # 首位需要判断的有：空格，符号，数字
        if head:
            if ch == ' ':
                continue
            elif ch == '+':
                head = False
                continue
            elif ch == '-':
                sign = False
                head = False
                continue
            elif ch.isdigit():
                head = False
                res = int(ch)
            else:
                break
        # 非首位只需判断数字，同时做越界判断
        else:    
            if ch.isdigit():
                res = res*10+int(ch)
                if sign and res > 2**31-1:
                    return 2**31-1
                if not sign and res > 2**31:
                    return -2**31
            else:
                break

    return res if sign else -res

0

[Valid Tic-Tac-Toe State](https://leetcode.com/problems/valid-tic-tac-toe-state/)。给出一个XO棋盘残局，判断该残局是否可能出现。条件：先手下X，后手下O。

思路：逻辑实现，注意几个非法条件即可。先手下X后手下O，那么'X'的数量肯定大于等于'O'，且差值最大为1；还有获胜条件，当'X'获胜时，'X'的数量肯定比'O'大1，因为'X'赢了之后'O'就不能再下了。

In [ ]:
def validTicTacToe(board: List[str]) -> bool:
    n_x = n_o = 0
    x_win = o_win = False

    for row in range(3):
        for col in range(3):
            if board[row][col] == 'X':
                n_x += 1
            if board[row][col] == 'O':
                n_o += 1

    if n_o > n_x or n_x > n_o+1:
        return False

    def win_detect(player):
        for row in range(3):
            if board[row][0] == board[row][1] == board[row][2] == player:
                return True
        for col in range(3):
            if board[0][col] == board[1][col] == board[2][col] == player:
                return True
        if board[0][0] == board[1][1] == board[2][2] == player:
            return True
        if board[0][2] == board[1][1] == board[2][0] == player:
            return True

    x_win, o_win = win_detect('X'), win_detect('O')
    if x_win and o_win:
        return False
    if x_win and n_x != n_o+1:
        return False
    if o_win and n_x != n_o:
        return False

    return True

[Remove Comments](https://leetcode.com/problems/remove-comments/)。移除C++程序中的注释。

思路：C++有两种注释，块注释```/*...*/```与行注释```//```，先出现的会覆盖后出现的。使用一个状态变量```block```记录是否存在块注释，那么需要分析四种情况。1. 不在块注释中，遇到```/*```，则进入注释区；2. 在块注释中，那么在遇到```*/```之前全部忽略；3. 不在块注释中，遇到```//```，忽略该行后所有字符；4. 不在任何注释中。

In [ ]:
def removeComments(source):
    block = False
    res = list()

    for line in source:
        i = 0
        if not block:
            val_line = list()    # 只有不在块注释区才会新建有效行
        while i < len(line):
            if line[i:i+2] == '/*' and not block:
                block = True
                i += 1    # 这里只+1是因为后面有个全局+1
            elif line[i:i+2] == '*/' and block:
                block = False
                i += 1
            elif not block and line[i:i+2] == '//':
                break    # 跳过当前行
            elif not block:
                val_line.append(line[i])    # 有效行增加字符
            i += 1

        if val_line and not block:
            res.append(''.join(val_line))

    return res

[Encode and Decode TinyURL](https://leetcode.com/problems/encode-and-decode-tinyurl/)。实现TinyURL的编码与解码。TinyURL是一种web服务，它将任意长URL转换成有固定前缀的短URL。

思路：使用哈希映射，对每一个URL都做编号。直接使用```dict```类型其实是一个取巧方法，该题其实是想考察哈希算法的实现。

In [ ]:
class Codec:
    def __init__(self):
        self.idx = 0
        self.table = dict()

    def encode(self, longUrl):
        """Encodes a URL to a shortened URL.

        :type longUrl: str
        :rtype: str
        """
        self.table[self.idx] = longUrl
        self.idx += 1
        return self.idx-1

    def decode(self, shortUrl):
        """Decodes a shortened URL to its original URL.

        :type shortUrl: str
        :rtype: str
        """
        return self.table[shortUrl]